<a href="https://www.kaggle.com/code/suneku/voice-changer-public?scriptVersionId=203548435" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### [w-okada's Voice Changer](https://github.com/deiteris/voice-changer) | **Kaggle**

---

## **⬇ VERY IMPORTANT ⬇**

You can use the following settings for optimal results:

Best performance: `f0: fcpe | Chunk: 56.0ms or higher | Extra: 2.7s`<br>
Best quality: `f0: rmvpe | Chunk: 64.0ms or higher | Extra: 5s`<br>
**Don't forget to select a GPU in the GPU field, <b>NEVER</b> use CPU!

You can tune `Chunk` for lower/higher delay and `Extra` for better quality.

---
**Credits**<br>
Realtime Voice Changer by [w-okada](https://github.com/w-okada)<br>
Original instructions by [Hina](https://github.com/HinaBl)<br>

**Need help?** [AI Hub Discord](https://discord.gg/aihub) » ***#help-realtime-vc***

---

# Kaggle Tutorial
Running this notebook can be a bit complicated.\
After created your Kaggle account, you'll need to **verify your phone number** to be able to use Internet Connection and GPUs.

Follow the instructions on the image below and don't forget to set **PERSISTENCE** to **Files only**.

![instructions.png](https://i.imgur.com/0NutkD8.png)

# Clone repository and install dependencies
This first step will download the latest version of Voice Changer and install the dependencies. **It will take some time to complete.**

In [ ]:
# This will make that we're on the right folder before installing
%cd /kaggle/working/

# To workaround an issue with METADATA not found for aiohttp
# !conda install -y aiohttp
!pip install colorama python-dotenv pyngrok --quiet

from colorama import Fore, Style
import requests, os

print(f"{Fore.CYAN}> Downloading prebuilt executable...{Style.RESET_ALL}")

def download_and_unpack(assets: list[dict]):
    for asset in assets:
        if not asset['name'].startswith('voice-changer-linux-amd64-cuda.tar.gz'):
            continue
        download_url = asset['browser_download_url']
        !wget -q --show-progress {download_url}

    print(f"{Fore.GREEN}> Unpacking...{Style.RESET_ALL}")
    !cat voice-changer-linux-amd64-cuda.tar.gz.* | tar xzf -
    print(f"{Fore.GREEN}> Finished unpacking!{Style.RESET_ALL}")
    !rm -rf voice-changer-linux-amd64-cuda.tar.gz.*

    !mv /kaggle/working/MMVCServerSIO/MMVCServerSIO /kaggle/working/MMVCServerSIO/VoiceChanger

def get_local_version(base: str) -> str:
    local_version = !cat {base}/_internal/version.txt
    return local_version[0].strip()

res = requests.get('https://api.github.com/repos/deiteris/voice-changer/releases/latest')
release_info = res.json()

remote_version = release_info['tag_name']
# Check and upgrade
if os.path.exists('MMVCServerSIO'):
    local_version = get_local_version("/kaggle/working/MMVCServerSIO")
    if remote_version != local_version:
        print(f"{Fore.CYAN}> Found new version. Current version {local_version}, remote version {remote_version}. Upgrading...{Style.RESET_ALL}")
        !rm -rf /kaggle/working/MMVCServerSIO/_internal /kaggle/working/MMVCServerSIO/VoiceChanger /kaggle/working/MMVCServerSIO/MMVCServerSIO
        download_and_unpack(release_info['assets'])
    else:
        print(f"{Fore.CYAN}> Current version {local_version} is the latest. Skipping download.{Style.RESET_ALL}")
else:
    print(f"{Fore.CYAN}> Downloading and installing version {remote_version}.{Style.RESET_ALL}")
    download_and_unpack(release_info['assets'])

%cd MMVCServerSIO

print(f"{Fore.CYAN}> Installing libportaudio2...{Style.RESET_ALL}")
!apt-get -y install libportaudio2 -qq
print(f"{Fore.CYAN}> Done! Proceed with the next steps.{Style.RESET_ALL}")

# Migrate the data

This cell will migrate your models from old version to new version (if present).

In [ ]:
import os
# Migrate old data to new version and delete old version
if os.path.exists("/kaggle/working/Hmod"):
    !mv /kaggle/working/Hmod/server/model_dir /kaggle/working/MMVCServerSIO/model_dir
    !mv /kaggle/working/Hmod/server/pretrain /kaggle/working/MMVCServerSIO/pretrain
    !rm -rf /kaggle/working/Hmod

# Configure the server

This cell will apply necessary configuration to the server.

In [ ]:
%cd /kaggle/working/MMVCServerSIO

from dotenv import set_key

set_key('.env', "SAMPLE_MODE", "")

Ready = True

# Start Server **using ngrok**
This cell will start the server, the first time that you run it will download the models, so it can take a while (~1-2 minutes)

---
You'll need a ngrok account, but <font color=green>**it's free**</font> and easy to create!
---
**1** - Create a **free** account at [ngrok](https://dashboard.ngrok.com/signup)\
**2** - If you didn't logged in with Google or Github, you will need to **verify your e-mail**!\
**3** - Click [this link](https://dashboard.ngrok.com/get-started/your-authtoken) to get your auth token, and replace **TOKEN_HERE** with your token.\
**4** - *(optional)* Change to a region near to you

In [ ]:
Token = 'TOKEN_HERE'
Region = "eu" # Read the instructions below
ClearConsole = True

# You can change the region for a better latency, use only the abbreviation
# Choose between this options:
# us -> United States (Ohio)
# ap -> Asia/Pacific (Singapore)
# au -> Australia (Sydney)
# eu -> Europe (Frankfurt)
# in -> India (Mumbai)
# jp -> Japan (Tokyo)
# sa -> South America (Sao Paulo)

# ---------------------------------
# DO NOT TOUCH ANYTHING DOWN BELOW!

%cd /kaggle/working/MMVCServerSIO

if not globals().get('Ready', False):
    print("Go back and run first and second cells.")
else:
    from pyngrok import conf, ngrok
    MyConfig = conf.PyngrokConfig()
    MyConfig.auth_token = Token
    MyConfig.region = Region[0:2]
    conf.get_default().authtoken = Token
    conf.get_default().region = Region
    conf.set_default(MyConfig);

    import threading, time, socket
    PORT = 18888

    import json
    from pyngrok import ngrok
    from IPython.display import clear_output

    ngrokConnection = ngrok.connect(PORT)
    public_url = ngrokConnection.public_url
    set_key('.env', "ALLOWED_ORIGINS", json.dumps([public_url]))

    def wait_for_server():
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', PORT))
            if result == 0:
                break
            sock.close()
        if ClearConsole:
            clear_output()
        print("--------- SERVER READY! ---------")
        print("Your server is available at:")
        print(public_url)
        print("---------------------------------")

    threading.Thread(target=wait_for_server, daemon=True).start()
    
    !chmod +x ./VoiceChanger
    
    !./VoiceChanger

    ngrok.disconnect(ngrokConnection.public_url)